In [ ]:
import sys
# sys.path.append('/root/kurly-aa-ds-demand-forecast')
import logging
import warnings
import numpy as np
import pandas as pd
from lightgbm import LGBMRegressor
from sklearn.model_selection import RandomizedSearchCV
from datetime import datetime, date, timedelta
from sklearn.preprocessing import LabelEncoder
import category_encoders as ce
warnings.filterwarnings("ignore")
from common.util.aargparser import get_aargparser, str2bool
from common.data.region_mapper import RegionMapper
from common.batch.dataprocessor import GCPDataProcessor
from common.io.read import AWSRead, GCPRead
from common.data.cookie import read_ar_goal_info_by_ondo_temp

In [ ]:
sql = """
select pdt
      , json_value(message.device_brand) as device_brand
      , json_value(message.device_id) as device_id
      , json_value(message.device_model) as device_model
      , json_value(message.event_id) as event_id
      , json_value(message.event_type) as event_type
      , json_value(message.event_properties.adid) as adid
      , json_value(message.event_properties.browse_id) as browse_id
      , json_value(message.event_properties.browse_screen_name) as browse_screen_name
      , json_value(message.event_properties.browse_site_name) as browse_site_name
      , json_value(message.event_properties.browse_tab_name) as browse_tab_name
      , json_value(message.event_properties.content_id) as content_id
      , json_value(message.event_properties.content_name) as content_name
      , json_value(message.event_properties.previous_screen_name) as previous_screen_name
      , json_value(message.event_properties.price) as price
      , json_value(message.event_properties.sales_price) as sales_price
      , json_value(message.event_properties.screen_name) as screen_naem
      , json_value(message.session_id) as session_id
      , json_value(message.timestamp) as timestamp
      , json_value(message.uuid) as uuid
      , upload_time
      , server_time
from `bq-marketkurly.kalog.kurly_app`
where pdt = '2023-09-01' and hr in (11)
"""

In [ ]:
# 유저정보
user_sql = """
select * 
from `bq-datafarm.data_warehouse.cust_target_master_1d`
where date(update_dt) = current_date('Asia/Seoul')
"""

In [ ]:
tot_sql = """
with user as (
  select * 
  from `bq-datafarm.data_warehouse.cust_target_master_1d`
  where date(update_dt) = current_date('Asia/Seoul')
)
, app_log as (
  select pdt
      , json_value(message.device_brand) as device_brand
      , json_value(message.device_id) as device_id
      , json_value(message.device_model) as device_model
      , json_value(message.event_id) as event_id
      , json_value(message.event_type) as event_type
      , json_value(message.event_properties.adid) as adid
      , json_value(message.event_properties.browse_id) as browse_id
      , json_value(message.event_properties.browse_screen_name) as browse_screen_name
      , json_value(message.event_properties.browse_site_name) as browse_site_name
      , json_value(message.event_properties.browse_tab_name) as browse_tab_name
      , json_value(message.event_properties.content_id) as content_id
      , json_value(message.event_properties.content_name) as content_name
      , json_value(message.event_properties.previous_screen_name) as previous_screen_name
      , json_value(message.event_properties.price) as price
      , json_value(message.event_properties.sales_price) as sales_price
      , json_value(message.event_properties.screen_name) as screen_naem
      , json_value(message.session_id) as session_id
      , json_value(message.timestamp) as timestamp
      , json_value(message.uuid) as uuid
      , upload_time
      , server_time
  from `bq-marketkurly.kalog.kurly_app`
  where pdt = '2023-09-01' and hr in (11)
)
SELECT A.*
    , B.CUST_NO
    , B.SEX
    , B.AGE_CATG
    , B.JOIN_DT
    , B.FIRST_ORD_DT
    , B.LAST_ORD_DT
    , B.LAST_ORD_TERM
    , B.ORD_SEQ
    , B.ORD_PAY_CUM_SUM
    , B.LAST_LOGIN_DT
    , B.SMS_YN
    , B.MAIL_YN
    , B.APP_PUSH_YN
    , B.APP_MKT_PUSH_YN
    , B.IS_KP_ACTIVE
    , B.CENTER_CD
    , B.REGION_GROUP_CD
    , B.LAST_CP_DC_RATE
    , B.TOT_CP_DC_RATE
    , B.POINT_BALANCE
    , B.BLACK_M_TYPE
    , B.RECOMMEND
    , B.MEMBERSHIP_YN
  FROM APP_LOG A 
  LEFT JOIN USER B 
  ON A.UUID = B.M_UUID
"""

In [ ]:
df = GCPRead().read_sql_query(sql)

In [ ]:
user = GCPRead().read_sql_query(user_sql)

In [ ]:
tot_df = GCPRead().read_sql_query(tot_sql)

In [ ]:
best_df = df.loc[df.browse_screen_name == 'popular_product'] # 베스트탭
best_df_market = best_df.loc[best_df.browse_site_name == 'market'] # 마켓 상품

In [ ]:
# 클릭수
best_df.loc[best_df.event_type.isin(['select_product', 'select_product_shortcut'])][['session_id', 'content_id', 'content_name']].drop_duplicates().groupby(['content_id', 'content_name']).count().sort_values(['session_id'], ascending = False)

In [ ]:
# 장바구니 ㄱ담ㄱ수
best_df.loc[best_df.event_type.isin(['select_cart'])].count().sort_values(['session_id'], ascending = False)